In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from tqdm import tqdm

root_dir = "../"
sys.path.append(root_dir)
import configs
import datasets
from ibydmt.utils.concepts import get_concepts
from ibydmt.utils.config import get_config
from ibydmt.utils.data import get_dataset
from ibydmt.utils.result import TestingResults
from ibydmt.tester import sweep, get_local_test_idx

config_name = "cub"
results_kw = {"testing.kernel_scale": 0.5, "testing.tau_max": 200}

config = get_config(config_name)
backbone_configs = sweep(config, sweep_keys=["data.backbone"])
dataset = get_dataset(config)
classes = dataset.classes

test_idx = get_local_test_idx(config)
n_test_images = sum([len(v) for v in test_idx.values()])

figure_dir = os.path.join(
    root_dir, "figures", config.name.lower(), "local_cond", "compare"
)
os.makedirs(figure_dir, exist_ok=True)

sns.set_style()
sns.set_context("paper")

In [ ]:
for i, backbone_config in enumerate(tqdm(backbone_configs)):
    backbone = backbone_config.data.backbone
    cardinalities = backbone_config.testing.cardinalities

    backbone_results = TestingResults.load(
        backbone_config, "local_cond", "image", results_kw=results_kw
    )

    f1_scores = np.zeros((len(cardinalities), n_test_images))
    i = 0
    for class_name, class_test_idx in test_idx.items():
        for idx in class_test_idx:
            _, concepts = get_concepts(config, concept_image_idx=idx)
            m = len(concepts)
            gt_importance = [1] * (m // 2) + [0] * (m // 2)

            for k, cardinality in enumerate(cardinalities):
                _, sorted_concepts, _, _, sorted_importance = backbone_results.sort(
                    class_name,
                    idx=idx,
                    cardinality=cardinality,
                    fdr_control=True,
                    with_importance=True,
                )

                importance = [
                    sorted_importance[concepts.index(c)] for c in sorted_concepts
                ]
                f1_scores[k, i] = metrics.f1_score(gt_importance, importance)

            i += 1

    f1_score_mu = np.mean(f1_scores, axis=1)
    f1_score_std = np.std(f1_scores, axis=1)
    print(
        f"{backbone}:"
        f" {' '.join([f'({mu:.2f}±{std:.2f})' for mu, std in zip(f1_score_mu, f1_score_std)])}"
    )